<a href="https://colab.research.google.com/github/jaybitrai/ML-Project-on-NYC-Taxi-dataset/blob/main/ML_Project_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    -



##### **Project Type**    - Regression
##### **Contribution**    - Individual/Team
##### **Team Member 1 -**  Jaykumar Bitrai
##### **Team Member 2 -**   
##### **Team Member 3 -**
##### **Team Member 4 -**

# **Project Summary -**

Write the summary here within 500-600 words.

# **GitHub Link -**

Provide your GitHub Link here.

https://github.com/jaybitrai/ML-Project-on-NYC-Taxi-dataset



# **Problem Statement**


**Write Problem Statement Here.**

To build a model that predicts the total ride durations of taxi trips in New York City

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required.
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits.
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule.

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
#Install Library  use to calculate the haversine distance

!pip install haversine

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
from numpy import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

from haversine import haversine

### Dataset Loading

In [ ]:
# Load Dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Importing the dataset
dataset = pd.read_csv('/content/drive/MyDrive/ALMABETTERPROJECTS/ML Project/Data/NYC Taxi Data.csv',encoding='latin1')

### Dataset First View

In [ ]:
# Dataset First Look
dataset.head()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
dataset.shape

### Dataset Information

In [ ]:
# Dataset Info
dataset.info()

In [ ]:
#Describe the dataset
dataset.describe()

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
len(dataset[dataset.duplicated()])

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
dataset.isna().sum()

In [ ]:
# Visualizing the missing values
sns.heatmap(dataset.isnull(), cbar=False)

### What did you know about your dataset?

Answer Here

We've given a dataset of taxi trip data which contains the geo co-ordinates of Both pickup point and drop point and the time of pickup and dropoff and the ride duration. For more inforation the dataset contains different variables such as vendor id and store and Fwd Flag

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
dataset.columns

In [ ]:
# Dataset Describe
dataset.describe(include='all')

### Variables Description

**id** - A unique identifier for each trip

**vendor_id** - A code indicating the provider associated with the trip rec

**pickup_datetime** - Date and time when the meter was engaged

**dropoff_datetime** - Date and time when the meter was disengaged

**passenger_count** - The number of passengers in the vehicle (driver entered value)

**pickup_latitude** - The latitude where the meter was engaged

**pickup_longitude** - The longitude where the meter was engaged

**dropoff_longitude** - The longitude where the meter was
disengaged

**dropoff_latitude** - The latitude where the meter was disengaged

**store_and_fwd_flag** - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip

**trip_duration** - Duration of the trip in seconds

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.
for i in dataset.columns.tolist():
  print("No. of unique values in ",i,"is",dataset[i].nunique(),".")

## **3**. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Write your code to make your dataset analysis ready.
#Creating a copy
df=dataset.copy()

In [ ]:
#Changing datetime column to datetime format
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

In [ ]:
#Avg Trip Duration
df['trip_duration'].mean()

In [ ]:
#Changing trip_duration column to numeric format
df['trip_duration'] = pd.to_numeric(df['trip_duration'])

In [ ]:
#Heatmap for correlation between individual Variables
sns.heatmap(df.corr(), annot=True)

In [ ]:
#Extracting date from DATETIME column
df['date'] = df['pickup_datetime'].dt.date

In [ ]:
#Extracting hour from DATETIME column
df['pickup_hour'] = df['pickup_datetime'].dt.hour
df['pickup_hour'].value_counts()

In [ ]:
#Extracting day name from DATETIME column
df['day_name'] = df['pickup_datetime'].dt.day_name()
df['day_name'].value_counts()

**Most of the bookings were done on Friday,Saturday,Thursday and as follows**

In [ ]:
df.columns

**The Busiest Hours for booking of the day are 18,19,21 and as follows**

In [ ]:
#Getting Avg trip duration according to days

df.groupby(['day_name'])['trip_duration'].mean()

**Trip duration is high on Friday, Monday, Saturday and as follows**

In [ ]:
#Creating empty column trip_duration_period
df['trip_duration_period'] = ''

In [ ]:
#Function to extract Type of trip duration

def get_period (time):
  if time <= 900:
    time_period = "Extremely Short Trip (less than 16 mins)"
  elif time < 1800:
    time_period = "Very Short Trip (less than half hour and greater than 15 mins)"
  elif time < 3600:
    time_period = "Short Trip (less than 1 hour and greater than half hour)"
  elif time < 7200:
   time_period = "Medium Trip (less than 2 hours and greater than 1 hour)"
  else:
    time_period = "Long Trip (greater than 2 hours)";
  return(time_period)

In [ ]:
#Creating column for type of trip duration
df['trip_duration_period'] = df.apply(lambda x:get_period(x['trip_duration']),axis=1)

In [ ]:
#Visualization of type of trip duration
df['trip_duration_period'].value_counts()

In [ ]:
#Extracting timing type of the day
def timing_distribution (hour):
  if hour>5 and hour<12:
    time_period = "Morning"
  elif hour>11 and hour<18:
    time_period = "Afternoon"
  elif hour>17 and hour<21:
     time_period = "Evening"
  else:
    time_period = "Night";
  return(time_period)

In [ ]:
#Creating column to extract timing type of the day
df['day_type'] = ''

In [ ]:
# extracting timing type of the day
df['day_type'] = df.apply(lambda x:timing_distribution(x['pickup_hour']),axis=1)

In [ ]:
#Visualization of timing type of the day
df['day_type'].value_counts()

In [ ]:
#calc_distance is a function to calculate distance between pickup and dropoff coordinates using Haversine formula.
def calc_distance(df):
    pickup = (df['pickup_latitude'], df['pickup_longitude'])
    drop = (df['dropoff_latitude'], df['dropoff_longitude'])
    return haversine(pickup, drop)

In [ ]:
#Calculate distance and assign new column to the dataframe.
df["distance"] = df.apply(lambda x: calc_distance(x),axis=1)

### What all manipulations have you done and insights you found?

**We've extracted the day name and created a column with it
To get more insights We've got more deep and got the pickup time by hour and creaeted a column for the timing type of day (Morning,Evening,Afternoon,Night)
We've also extracted the type of trip duration based on the trip duration
Visualized the correlation using heatmap**

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1 Bar Chart for **Pickup hour vs Bookings**

In [ ]:
# Chart - 1 visualization code
# Create the figure object
ax = df['pickup_hour'].value_counts().plot.bar(
    figsize = (8,5),
    fontsize = 10
)

# Set the title
ax.set_title("Number of bookings by hour", fontsize = 15)

# Set x and y-labels
ax.set_xlabel("Pickup hour", fontsize = 10)
ax.set_ylabel("Number of bookings", fontsize = 10)

##### 1. Why did you pick the specific chart?

**Because It has more number of X Values
It is good to visualize in bar chart**

##### 2. What is/are the insight(s) found from the chart?

**The maximum number of bookings were done on 18,19,21 and as follows hour of the day**

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

**Helps in creating positive business impact as we can see there is more demand on the 18,19 and 21 hour of the day**

#### Chart - 2 Pickup Day vs Bookings

In [ ]:
# Chart - 2 visualization code
# Create the figure object
ax = df['day_name'].value_counts().plot.bar(
    figsize = (8,5),
    fontsize = 10
)

# Set the title
ax.set_title("Number of bookings by day", fontsize = 15)

# Set x and y-labels
ax.set_xlabel("Pickup day", fontsize = 10)
ax.set_ylabel("Number of bookings", fontsize = 10)

##### 1. Why did you pick the specific chart?

**I'd extracted the bookings according the days of week and it would be more easy to visualize using bar chart**

##### 2. What is/are the insight(s) found from the chart?

**Maximum number of bookings were done on Friday, Saturday and Thursday.
But we can see here, there is not much difference on least and the most number of booking by day**

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

**Yes the gained insight will create a positive business impact as we can see there is more demand on Weekends**

#### Chart - 3 Pie Chart by **Vendor ID**

In [ ]:
# Chart - 3 visualization code
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.2f}%  ({v:d})'.format(p=pct,v=val)
    return my_autopct
df['vendor_id'].value_counts().plot(kind='pie',
                              figsize=(15,6),
                               autopct=make_autopct(list(df['vendor_id'].value_counts())),
                               labels=['Vendor_id=2','Vendor_id=1'],
                               colors=['skyblue','red'],
                               explode=[0,0]
                              )

##### 1. Why did you pick the specific chart?

**There were only two values of vendor_id so it is better to visuaize it on pie chart and easy to gain insights. We've done to check each vendor distribution**

##### 2. What is/are the insight(s) found from the chart?

**The more number of bookings were done by vendor id 2 but we can see there is no much difference within them**

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

**Yes. We can see that vendor 2 has done more bookings than vendor 1**

#### Chart - 4 Bar Chart by **Passenger Count**

In [ ]:
# Chart - 4 visualization code
ax = df['passenger_count'].value_counts().plot(kind='bar',figsize=(8,5))

#variance = 0.2 Million

# Set the title
ax.set_title("Passenger Count per booking", fontsize = 15)

# Set x and y-labels
ax.set_xlabel("Number of Passenger count", fontsize = 10)
ax.set_ylabel("Number of Bookings", fontsize = 10)

##### 1. Why did you pick the specific chart?

**We've done this to visualize the number of passenger count by booking (i.e no of passenger in one booking) It has more values so we'd choose bar chart**

##### 2. What is/are the insight(s) found from the chart?

**We can clearly see here that number of maximum bookings had passenger count of 1 and 2. We can also see that barely minimum number of bookings were done with passenger count 7 8 and 9**

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

**Yes. We can see that many bookings were done by 1 and 2 passenger count.
So we can use the 3 or 4 seater vehicle more**

#### Chart - 5 Bar Chart for **duration** *period wise*

In [ ]:
# Chart - 5 visualization code
ax = df['trip_duration_period'].value_counts().plot(kind='bar',figsize=(8,6))

ax.set_title ("Trip Time Durations", fontsize = 10)

ax.set_xlabel("Trip Durations", fontsize = 10)
ax.set_ylabel("Number of Bookings", fontsize = 10)


##### 1. Why did you pick the specific chart?

**We've extracted the type of trip duration based on the trip duration
We've used the bar chart because it was easy to visualize according to bar chart**

##### 2. What is/are the insight(s) found from the chart?

**Most of the bookings were done for the short time.
As we can see nearly 80% of bookings were done for less than 30 mins**

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

**Yes. I'll have a positive impact as we can see there is high demand for short trip duration we can go accordingly**

#### Chart - 6 Pie Chart **Store and Fwd Flag**

In [ ]:
# Chart - 6 visualization code
df['store_and_fwd_flag'].value_counts().plot(kind='pie',
                              figsize=(20,10),
                               autopct=make_autopct(list(df['store_and_fwd_flag'].value_counts())),
                               colors=['yellow','red'],
                               explode=[0,0]
                              )

##### 1. Why did you pick the specific chart?

**As there were only 2 values, It is good to visualize using pie chart**

##### 2. What is/are the insight(s) found from the chart?

**The most bookings data weren't stored and forwarded.
Only less than 1% were stored and forwarded**

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

**No impact**

#### Chart - 7 Pie Chart Distribution by Day type

In [ ]:
# Chart - 7 visualization code
df['day_type'].value_counts().plot(kind='pie',
                              figsize=(8,10),
                               autopct=make_autopct(list(df['day_type'].value_counts())),
                               colors=['blue','red','yellow','green']
                              )

##### 1. Why did you pick the specific chart?

**As there were less values so It was better to visualize using pie chart
We've created the distribution of day type by bookings**

##### 2. What is/are the insight(s) found from the chart?

**As we can see, We had the overall demand same for every period of the day.
Only the evening period was having low demand comparatively**

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

**Yes. We've to keep working the portal overall as we have booking of all the type of day**

## ***5. Hypothesis Testing***

### Based on your chart experiments, define three hypothetical statements from the dataset. In the next three questions, perform hypothesis testing to obtain final conclusion about the statements through your code and statistical testing.

Answer Here.

### Hypothetical Statement - 1

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

### Hypothetical Statement - 2

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

### Hypothetical Statement - 3

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

## ***6. Feature Engineering & Data Pre-processing***

### 1. Handling Missing Values

**No Missing values were found**

#### What all missing value imputation techniques have you used and why did you use those techniques?

Answer Here.

### 2. Handling Outliers

In [ ]:
# Handling Outliers & Outlier treatments
plt.figure(figsize=(15,8))
plt.title("Box plot of  distance ")
ax = sns.boxplot(data=df['distance'], orient="v")

In [ ]:
percentile_q1 = np.percentile(df['distance'],25)
print(percentile_q1)
percentile_q2 = np.percentile(df['distance'],50)
print(percentile_q2)
percentile_q3 = np.percentile(df['distance'],75)
print(percentile_q3)

In [ ]:
iqr=percentile_q3 - percentile_q1
lower_limit_outlier=percentile_q1-1.5*iqr
upper_limit_outlier=percentile_q3+1.5*iqr

print("lower limit for outlier  :",lower_limit_outlier)
print("Upper limit for outlier  :",upper_limit_outlier)

In [ ]:
df=df[df['distance']>lower_limit_outlier]
df=df[df['distance']<upper_limit_outlier]

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(15,8))
plt.title("Box plot of  trip_duration ")
ax = sns.boxplot(data=df['trip_duration'], orient="v")

In [ ]:
percentile_q1_trip_duration = np.percentile(df['trip_duration'],25)
print(percentile_q1_trip_duration)
percentile_q2_trip_duration = np.percentile(df['trip_duration'],50)
print(percentile_q2_trip_duration)
percentile_q3_trip_duration = np.percentile(df['trip_duration'],75)
print(percentile_q3_trip_duration)

In [ ]:
iqr=percentile_q3_trip_duration - percentile_q1_trip_duration
lower_limit_outlier_trip_duration=percentile_q1_trip_duration-1.5*iqr
upper_limit_outlier_trip_duration=percentile_q3_trip_duration+1.5*iqr

print("lower limit for outlier  :",lower_limit_outlier_trip_duration)
print("Upper limit for outlier  :",upper_limit_outlier_trip_duration)

In [ ]:
df=df[df['trip_duration']>0]
df=df[df['trip_duration']<upper_limit_outlier_trip_duration]

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(15,8))
plt.title("Box plot of  passenger_count ")
ax = sns.boxplot(data=df['passenger_count'])

In [ ]:
percentile_q1_passenger_count = np.percentile(df['passenger_count'],25)
print(percentile_q1_passenger_count)
percentile_q2_passenger_count = np.percentile(df['passenger_count'],50)
print(percentile_q2_passenger_count)
percentile_q3_passenger_count = np.percentile(df['passenger_count'],75)
print(percentile_q3_passenger_count)

In [ ]:
iqr=percentile_q3_passenger_count - percentile_q1_passenger_count
lower_limit_outlier_passenger_count=percentile_q1_passenger_count-1.5*iqr
upper_limit_outlier_passenger_count=percentile_q3_passenger_count+1.5*iqr

print("lower limit for outlier  :",lower_limit_outlier_passenger_count)
print("Upper limit for outlier  :",upper_limit_outlier_passenger_count)

In [ ]:
nyc_df=df[df['passenger_count']>0]
nyc_df=df[df['passenger_count']<upper_limit_outlier_passenger_count]

In [ ]:
df.shape

##### What all outlier treatment techniques have you used and why did you use those techniques?

Outlier Detection using IQR Method
IQR=Q3-Q1

lower_limit_outlier=Q1-1.5*IQR

upper_limit_outlier=Q3+1.5*IQR¶

so we have to take the values which is greater then lower limit outlier and less then upper limit outlier remove the outlier present in distance



### 3. Categorical Encoding

In [ ]:
# Encode your categorical columns
df = pd.get_dummies(df,columns=['store_and_fwd_flag','day_name','trip_duration_period','day_type','vendor_id','passenger_count'],drop_first=False)

#### What all categorical encoding techniques have you used & why did you use those techniques?

Answer Here.

### 4. Textual Data Preprocessing
(It's mandatory for textual dataset i.e., NLP, Sentiment Analysis, Text Clustering etc.)

#### 1. Expand Contraction

In [ ]:
# Expand Contraction

#### 2. Lower Casing

In [ ]:
# Lower Casing

#### 3. Removing Punctuations

In [ ]:
# Remove Punctuations

#### 4. Removing URLs & Removing words and digits contain digits.

In [ ]:
# Remove URLs & Remove words and digits contain digits

#### 5. Removing Stopwords & Removing White spaces

In [ ]:
# Remove Stopwords

In [ ]:
# Remove White spaces

#### 6. Rephrase Text

In [ ]:
# Rephrase Text

#### 7. Tokenization

In [ ]:
# Tokenization

#### 8. Text Normalization

In [ ]:
# Normalizing Text (i.e., Stemming, Lemmatization etc.)

##### Which text normalization technique have you used and why?

Answer Here.

#### 9. Part of speech tagging

In [ ]:
# POS Taging

#### 10. Text Vectorization

In [ ]:
# Vectorizing Text

##### Which text vectorization technique have you used and why?

Answer Here.

### 4. Feature Manipulation & Selection

#### 1. Feature Manipulation

In [ ]:
# Manipulate Features to minimize feature correlation and create new features

#### 2. Feature Selection

In [ ]:
df.columns

In [ ]:
# Select your features wisely to avoid overfitting
features = [ 'vendor_id_1', 'vendor_id_2', 'passenger_count_0',
       'passenger_count_1', 'passenger_count_2', 'passenger_count_3',
       'passenger_count_4', 'passenger_count_5', 'passenger_count_6',
       'passenger_count_7', 'passenger_count_8', 'passenger_count_9',
       'pickup_hour', 'distance', 'store_and_fwd_flag_N',
       'store_and_fwd_flag_Y', 'day_name_Friday', 'day_name_Monday',
       'day_name_Saturday', 'day_name_Sunday', 'day_name_Thursday',
       'day_name_Tuesday', 'day_name_Wednesday',
       'trip_duration_period_Extremely Short Trip (less than 16 mins)',
       'trip_duration_period_Very Short Trip (less than half hour and greater than 15 mins)',
       'day_type_Afternoon', 'day_type_Evening', 'day_type_Morning',
       'day_type_Night']

##### What all feature selection methods have you used  and why?

I've used the features that are having relation with the total trip duration timing

##### Which all features you found important and why?

**No Correlation was found**

### 5. Data Transformation

#### Do you think that your data needs to be transformed? If yes, which transformation have you used. Explain Why?

In [ ]:
# Transform Your data

### 6. Data Scaling

In [ ]:
# Scaling your data

##### Which method have you used to scale you data and why?

### 7. Dimesionality Reduction

##### Do you think that dimensionality reduction is needed? Explain Why?

Answer Here.

In [ ]:
# DImensionality Reduction (If needed)

##### Which dimensionality reduction technique have you used and why? (If dimensionality reduction done on dataset.)

Answer Here.

### 8. Data Splitting

In [ ]:
# Split your data to train and test. Choose Splitting ratio wisely.

##### What data splitting ratio have you used and why?

Answer Here.

### 9. Handling Imbalanced Dataset

##### Do you think the dataset is imbalanced? Explain Why.

Answer Here.

In [ ]:
# Handling Imbalanced Dataset (If needed)

##### What technique did you use to handle the imbalance dataset and why? (If needed to be balanced)

Answer Here.

## ***7. ML Model Implementation***

### ML Model - 1 **Linear Regression**

In [ ]:
y = np.log10(df['trip_duration'])

In [ ]:
df[features].head()

In [ ]:
df.columns

In [ ]:
from scipy.stats import zscore
#Train test split
X = df[features].apply(zscore)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X,y , test_size = 0.2, random_state = 0)
print(X_train.shape)
print(X_test.shape)

In [ ]:
# ML Model - 1 Implementation
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X_train, y_train)

In [ ]:
reg.score(X_train, y_train)

In [ ]:
reg.coef_

In [ ]:
y_pred = reg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

MSE  = mean_squared_error(10**(y_test), 10**(y_pred))
print("MSE :" , MSE)

RMSE = np.sqrt(MSE)
print("RMSE :" ,RMSE)

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(10**(y_test), 10**(y_pred))
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(10**(y_test), 10**(y_pred)))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(10**(y_pred))
plt.plot(np.array(10**(y_test)))
plt.legend(["Predicted","Actual"])
plt.show()

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### ML Model - 2 **Lasso Regression**

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
from sklearn.linear_model import Lasso
lasso  = Lasso(alpha=0.1 , max_iter= 3000)

lasso.fit(X_train, y_train)

In [ ]:
lasso.score(X_train, y_train)

In [ ]:
y_pred_l = lasso.predict(X_test)

In [ ]:
MSE  = mean_squared_error(10**(y_test), 10**(y_pred_l))
print("MSE :" , MSE)

RMSE = np.sqrt(MSE)
print("RMSE :" ,RMSE)

r2 = r2_score(10**(y_test), 10**(y_pred_l))
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(10**(y_test), 10**(y_pred_l)))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)
from sklearn.model_selection import GridSearchCV


In [ ]:
### Cross validation
lasso = Lasso()
parameters = {'alpha': [1e-15,1e-13,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1,5,10,20,30,40,45,50,55,60,100]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv=3)
lasso_regressor.fit(X_train, y_train)

In [ ]:
print("The best fit alpha value is found out to be :" ,lasso_regressor.best_params_)
print("\nUsing ",lasso_regressor.best_params_, " the negative mean squared error is: ", lasso_regressor.best_score_)

In [ ]:
y_pred_lasso = lasso_regressor.predict(X_test)

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(10**(y_pred_lasso))
plt.plot(10**(np.array(y_test)))
plt.legend(["Predicted","Actual"])
plt.show()

In [ ]:
MSE  = mean_squared_error(10**(y_test), 10**(y_pred_lasso))
print("MSE :" , MSE)

RMSE = np.sqrt(MSE)
print("RMSE :" ,RMSE)

r2 = r2_score(10**(y_test), 10**(y_pred_lasso))
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(10**(y_test), 10**(y_pred_lasso)))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

#### 3. Explain each evaluation metric's indication towards business and the business impact pf the ML model used.

Answer Here.

### ML Model - 3

In [ ]:
# ML Model - 3 Implementation

# Fit the Algorithm

# Predict on the model

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 3 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### 1. Which Evaluation metrics did you consider for a positive business impact and why?

Answer Here.

### 2. Which ML model did you choose from the above created models as your final prediction model and why?

Answer Here.

### 3. Explain the model which you have used and the feature importance using any model explainability tool?

Answer Here.

## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


In [ ]:
# Load the File and predict unseen data.

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***